# Task 5 - Train User-specific Classifiers

In [1]:
import math
import random

import scipy.stats as stats
import numpy as np
import pandas as pd
import mne

import matplotlib.pyplot as plt
import seaborn as sns

from swlda import SWLDA
from scipy.stats import norm

import utils

%load_ext autoreload
%autoreload 2

In [91]:
NUM_TIMESTAMPS = 195
EPOCH_SIZE = 15
NUM_SESSIONS = 6
directory = '/Users/zionshane/Desktop/Duke/Research/BCI_data/EDFData-StudyA'
obj = 1
obj = str(obj) if obj > 10 else '0'+str(obj)
paradigm = 'RD'
obj_directory = directory + f'/A{obj}/SE001'

In [92]:
train_features,train_response = utils.load_data(dir=obj_directory,
                                                obj=obj,
                                                num_timestamps=NUM_TIMESTAMPS,
                                                epoch_size=EPOCH_SIZE,
                                                type=paradigm,
                                                mode='train',
                                                num_sessions=NUM_SESSIONS)

In [93]:
test_features,test_response = utils.load_data(dir=obj_directory,
                                              obj=obj,
                                              num_timestamps=NUM_TIMESTAMPS,
                                              epoch_size=EPOCH_SIZE,
                                              type=paradigm,
                                              mode='test',
                                              num_sessions=NUM_SESSIONS)

In [94]:
clf = SWLDA(penter=0.1, premove=0.15)
clf.fit(train_features, train_response)

In [95]:
auc = clf.test(test_features, test_response)
auc

0.6314880795805274

## Archived code

In [7]:
train_epochs_list = []
for i in range(1, 6):
    i = str(i) if i >= 10 else '0'+str(i)
    train_file = directory+f'/A{obj}/SE001/Train/RC/A{obj}_SE001RC_Train{i}.edf'
    raw_train_data = mne.io.read_raw_edf(train_file, preload=True, verbose=False)
    eeg_channels = mne.pick_channels_regexp(raw_train_data.info['ch_names'], 'EEG')
    raw_train_data.notch_filter(freqs=60, picks=eeg_channels, verbose=False)
    part_train_epochs = utils.get_core_epochs(raw_train_data)
    train_epochs_list.append(part_train_epochs)

In [8]:
train_features = np.array([], dtype=np.float64)
train_response = np.array([], dtype=np.float64)
for train_epochs in train_epochs_list:
    features, response = utils.split_data(train_epochs,
                                          n_times=NUM_TIMESTAMPS,
                                          n_samples=EPOCH_SIZE)
    # I follow this stackoverflow post to concatenate np.array
    # link: https://stackoverflow.com/a/22732845/22322930
    train_features = np.concatenate([train_features, features]) if train_features.size else features
    train_response = np.concatenate([train_response, response]) if train_response.size else response

In [9]:
test_epochs_list = []
for i in range(6, 11):
    i = str(i) if i >= 10 else '0'+str(i)
    test_file = directory+f'/A{obj}/SE001/Test/RC/A{obj}_SE001RC_Test{i}.edf'
    raw_test_data = mne.io.read_raw_edf(test_file, preload=True, verbose=False)
    eeg_channels = mne.pick_channels_regexp(raw_test_data.info['ch_names'], 'EEG')
    raw_test_data.notch_filter(freqs=60, picks=eeg_channels, verbose=False)
    part_test_epochs = utils.get_core_epochs(raw_test_data)
    test_epochs_list.append(part_test_epochs)

In [10]:
test_features = np.array([], dtype=np.float64)
test_response = np.array([], dtype=np.float64)
for test_epochs in test_epochs_list:
    features, response = utils.split_data(test_epochs,
                                          n_times=NUM_TIMESTAMPS,
                                          n_samples=EPOCH_SIZE)
    test_features = np.concatenate([test_features, features]) if test_features.size else features
    test_response = np.concatenate([test_response, response]) if test_response.size else response

In [11]:
clf = SWLDA(penter=0.1, premove=0.15)
clf.fit(train_features, train_response)

In [12]:
auc = clf.test(test_features, test_response)
auc

0.7023992501128536